http://www.geo-ref.net/ph/bel.htm

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
# Load the shapefile
vectors = gpd.read_file("/Users/camillecu/Downloads/KUL/MDA/project/clustering/vector_shape/be.shp")
# intervention is parquet file
intervention = pd.read_parquet("/Users/camillecu/Downloads/KUL/MDA/project/preprocess_intervention/intvn_clean.parquet")

cad9 = pd.read_parquet("/Users/camillecu/Downloads/KUL/MDA/project/preprocess_intervention/cad9_clean.parquet")

In [9]:
vectors.head()

,mun_code,smun_code,prov_name_,arr_name_n,mun_name_n,aed_count,ambulance_,pit_count,mug_count,vectors,geometry
0,31005,31005G,Provincie West-Vlaanderen,Arrondissement Brugge,Brugge,119.0,2.0,0.0,1.0,122.0,"POLYGON ((3.21386 51.16520, 3.19908 51.16144, ..."
1,31006,31006-Z,Provincie West-Vlaanderen,Arrondissement Brugge,Damme,2.0,0.0,0.0,0.0,2.0,"POLYGON ((3.30594 51.22328, 3.30363 51.22391, ..."
2,31012,31012E,Provincie West-Vlaanderen,Arrondissement Brugge,Jabbeke,7.0,1.0,0.0,0.0,8.0,"POLYGON ((3.09117 51.14577, 3.09044 51.14634, ..."
3,31040,31040D,Provincie West-Vlaanderen,Arrondissement Brugge,Zedelgem,12.0,0.0,0.0,0.0,12.0,"POLYGON ((3.06641 51.10243, 3.06618 51.10360, ..."
4,31043,31043B,Provincie West-Vlaanderen,Arrondissement Brugge,Knokke-Heist,25.0,1.0,0.0,1.0,27.0,"POLYGON ((3.26466 51.32048, 3.26421 51.32104, ..."


In [11]:
intervention.head()

,mission id,service name,postalcode permanence,cityname permanence,streetname permanence,latitude permanence,longitude permanence,permanence short name,permanence long name,vector type,...,streetname destination hospital,calculated traveltime destinatio,calculated distance destination,number of transported persons,abandon reason,time period,time intervention,eventlevel firstcall percentage,same event type,smae event level
0,10221520001,HA UR MECH AZ St Maarten,2800.0,Mechelen (Mechelen),Liersesteenweg,51.05102,4.47803,AAMECH01A,ZW MECHELEN 1,Ambulance,...,None,NaN,NaN,NaN,Verzorgd ter plaatse,midnight,26.0,79.033900,1,1
1,10221520002,HA UR ANTW Stuivenberg,2060.0,Antwerpen (Antwerpen),Lange Beeldekensstraat,51.22249,4.43629,UAANTW01A,MUG ANTWERPEN 1,MUG,...,None,NaN,NaN,NaN,None,midnight,63.0,79.033900,1,0
2,10221520004,BA ANTW [Borgerhout] AMBUCE,2140.0,Antwerpen (Borgerhout),Gijselsstraat,51.21562,4.44392,AAANTW07A,ZW ANTWERPEN 7,Ambulance,...,None,NaN,NaN,NaN,Zonder patient,midnight,32.0,79.033900,1,1
3,10221520005,BA WIJN AMBUCE,2110.0,Wijnegem (Wijnegem),Bijkhoevelaan,51.23355,4.49318,AAWIJN01A,ZW WIJNEGEM 1,Ambulance,...,Lange Bremstraat,530.0,5468.0,1.0,None,midnight,72.0,79.033900,1,1
4,10221520006,FA HVP BEER Taxandria,2340.0,Beerse (Beerse),Bisschopslaan,51.31482,4.85623,AABEER01A,ZW BEERSE 1,Ambulance,...,Rubensstraat,859.0,13231.0,1.0,None,midnight,49.0,0.831988,1,1


P039 - Cardiac problem (other than thoracic pain)

In [11]:
intervention.columns

Index(['service name', 'postalcode permanence', 'latitude permanence',
       'longitude permanence', 'vector type', 'eventtype firstcall',
       'eventlevel firstcall', 'eventtype trip', 'eventlevel trip',
       'latitude intervention', 'longitude intervention',
       'province intervention', 't0', 't6', 'waiting time',
       'intervention duration', 'departure time (t1reported)',
       'departure time (t1confirmed)', 'calculated traveltime destinatio',
       'calculated distance destination', 'number of transported persons',
       'time period', 'time intervention', 'eventlevel firstcall percentage',
       'same event type', 'smae event level'],
      dtype='object')

In [10]:
# drop the columns that are not needed:
#  mission id, cityname permanence, streetname permanence, 'permanence short name', 'permanence long name','postalcode intervention'
# 'cityname intervention', 'name destination hospital','cityname destination hospital', 'streetname destination hospital', 'abandon reason'
intervention = intervention.drop(columns=['mission id', 'cityname permanence', 'streetname permanence', 'permanence short name', 
                                          'permanence long name', 'postalcode intervention', 'cityname intervention', 
                                          'name destination hospital', 'cityname destination hospital', 'streetname destination hospital', 
                                          'abandon reason'])

In [13]:
interv_event_level = intervention[['eventlevel firstcall', 'eventlevel trip', 'latitude intervention', 'longitude intervention',
    'province intervention', 'eventlevel firstcall percentage', 'smae event level']]

interv_event_type = intervention[['eventtype firstcall', 'eventtype trip', 'latitude intervention', 'longitude intervention','same event type']]

interv_time = intervention[['t0', 't6', 'waiting time',
       'intervention duration', 'departure time (t1reported)',
       'departure time (t1confirmed)', 'calculated traveltime destinatio',
       'calculated distance destination', 'number of transported persons',
       'time period', 'time intervention']]

interv_location = intervention[['latitude intervention', 'longitude intervention', 'latitude permanence', 'longitude permanence']]

# interv_event_type

In [19]:
# frequency of 'same event type' in interv_event_type
interv_event_type['same event type'].value_counts()

same event type
1    320520
0     17967
Name: count, dtype: int64

### which type was most frequently mislabel? 

In [21]:
# list distinct values of eventtype firstcall
# interv_event_type['eventtype firstcall'].unique()
# interv_event_type['eventtype trip'].unique()

# interv_event_level

In [20]:
# frequency of 'smae event level' in interv_event_level
interv_event_level['smae event level'].value_counts()

smae event level
1    320298
0     18189
Name: count, dtype: int64

### which region has more severe cases?